In [ ]:
import streamlit as st
import speech_recognition as sr
from wit import Wit
import io

st.title('Real-time Speech-to-Text Transcription App')

# Choose the API for speech recognition
api_selection = st.radio("Choose the Speech Recognition API", ('Google Speech Recognition', 'Wit.ai'))

# Choose the language
language_option = st.radio("Choose the Language", ('English', 'French', 'Spanish', 'German', 'Chinese'))

language_codes = {'English': 'en-US', 'Spanish': 'es-ES', 'French': 'fr-FR', 'German': 'de-DE', 'Chinese': 'zh-CN'}

def recognize_speech_from_mic(recognizer, microphone, selected_api, language):
    st.write("Please speak into the microphone")
    with microphone as source:
        audio = recognizer.listen(source)
    st.write("Got it! Now to recognize it...")

    # Recognize the speech using the chosen API
    if selected_api == 'Google Speech Recognition':
        try:
            transcription = recognizer.recognize_google(audio)
        except sr.RequestError:
            # API was unreachable or unresponsive
            transcription = "API unavailable/unresponsive"
        except sr.UnknownValueError:
            # speech was unintelligible
            transcription = "Unable to recognize speech"

    elif api_selection == 'Wit.ai':
        audio_bytes = io.BytesIO(audio.get_wav_data()).read()
        client = Wit('QQKBFTRECWO23JNY32PRZOLI5THU7L6Z')
        resp = client.speech(audio_bytes, {'Content-Type': 'audio/wav'})
        if 'text' in resp:
            transcription = resp['text']
        else:
            transcription = "Could not extract text from the response."
    return transcription



# Click the button to start recording and recognition
if st.button('Start Recording'):
    recognizer = sr.Recognizer()
    microphone = sr.Microphone()

    transcription = recognize_speech_from_mic(recognizer, microphone, api_selection, language_codes[language_option])
    st.write('Transcription:')
    st.write(transcription)
